In [1]:
import csv

spots = {role: int(n) for role, n in csv.reader(open('spots.csv'))}
all_rankings = [ranking for ranking in csv.DictReader(open('rankings.csv'))]

In [2]:
by_role, by_person = {}, {}

for ranking in all_rankings:
    by_role.setdefault(ranking['role'], []).append(ranking)
    by_person.setdefault(ranking['person'], []).append(ranking)

In [3]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()
solver = cp_model.CpSolver()

In [4]:
for ranking in all_rankings:
    ranking['selected'] = model.NewBoolVar('')

In [5]:
for role, rankings in by_role.items():
    total = sum(ranking['selected'] for ranking in rankings)
    model.Add(total <= spots[role])

In [6]:
for person, rankings in by_person.items():
    total = sum(ranking['selected'] for ranking in rankings)
    model.Add(total <= 1)

In [7]:
total_score = 0
for ranking in all_rankings:
    score = float(ranking['score']) * ranking['selected']
    total_score += score

In [8]:
model.Maximize(total_score)
status = solver.Solve(model)
print(solver.Value(total_score), solver.StatusName(status))

82.87644300144302 OPTIMAL


In [9]:
with open('results.csv', 'w') as f:
    writer = csv.DictWriter(f, all_rankings[0].keys())
    writer.writeheader()
    for ranking in all_rankings:
        ranking['selected'] = solver.Value(ranking['selected'])
        writer.writerow(ranking)